In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

In [6]:
# Load configurations
config = read_params(config_path)
project_dir = config["project_dir"]
scored_limits_risk_review_data_path_excel_parquet = config["processed_data_config"]["scored_limits_risk_review_data_excel_parquet"]
before_21d_graduation_limits_data_path = config["interim_data_config"]["before_21d_graduation_limits_data_parquet"]
after_rmdd_ge95_data_path = config["interim_data_config"]["after_rmdd_ge95_data_parquet"]
after_rmdd_ge96_data_path = config["interim_data_config"]["after_rmdd_ge96_data_parquet"]
sr_scored_limits_approved_data_path = config["processed_data_config"]["sr_scored_limits_approved_data_parquet"]
test_accounts = config["test_accounts_config"]["store_number"]

### Limit Investigation

In [7]:
# Columns in scope
limit_cols = ['final_21_limit', 'final_7_limit', 'final_1_limit']

#### Ubuntu 2.0

In [8]:
# Ubuntu 2.0 refresh run
df_sr_scored_limits_approved = pd.read_parquet(project_dir + sr_scored_limits_approved_data_path)
df_sr_scored_limits_approved.sample(2)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,model_version,created_at,max_global_limit,previous_max_global_limit,max_limit_diff,rules_summary_narration,limit_reason,record_added_to_warehouse_on_timestamp
26294,254728780762,605220,2022-09-20,2022-10-20,29.00,149968.39,31.00,0.94,No_rules_relaxed,27.00,No,0.00,21147671,True,Reject,0.00,254728780762,23.00,300.00,21.00,110000.00,0.00,2022-07-29,2022-08-19,NaT,2022-08-19,2022-08-24,2022-11-22,89.00,2.00,active_default,476.00,110000.00,2022-07-29,21.00,0.91,110.00,0.00,89.00,1.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,84.00,2160900.00,948200.00,946200.00,0.00,0.00,0.00,1.00,1.00,1.00,23.00,0.35,0.12,0.12,0.70,0.71,0.71,Band 7,Band 8,0.55,0.30,0.30,0.00,1.00,0.00,0.25,0.10,0.05,0.05,14434.46,7939.50,7939.50,0.00,0.00,0.00,0,0,1.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-11-17]",2022-11-17 12:16:47,0.00,0.00,0.00,bad repayment history,A2,2022-11-17 12:17:04:747376
21016,254726405434,7764353,NaT,NaT,0.00,29823.42,0.00,0.00,No_rules_relaxed,-4.00,0,0.00,24731977,True,Reject,0.00,254726405434,3.00,600.00,7.00,60000.00,60000.00,2022-06-20,2022-06-27,2022-09-07,2022-06-27,2022-06-30,2022-09-28,72.00,2.00,closed_default,472.00,60000.00,2022-06-20,2.00,0.67,149.00,0.00,72.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,69.00,0.00,1464700.00,1488200.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.35,0.12,0.12,0.70,0.71,0.71,Band 1,Band 3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-11-17]",2022-11-17 12:16:47,0.00,0.00,0.00,bad repayment history,A2,2022-11-17 12:17:04:747376


In [9]:
# Data set dimension
df_sr_scored_limits_approved.shape

(139160, 114)

In [10]:
# Data set schema
df_sr_scored_limits_approved.dtypes.to_frame().T

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,model_version,created_at,max_global_limit,previous_max_global_limit,max_limit_diff,rules_summary_narration,limit_reason,record_added_to_warehouse_on_timestamp
0,object,object,object,object,float64,float64,float64,float64,object,float64,object,float64,object,object,object,float64,object,float64,float64,float64,float64,float64,object,object,object,object,datetime64[ns],datetime64[ns],float64,float64,object,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object


In [11]:
# Legacy limits
df_sr_scored_limits_approved[limit_cols].sum()

final_21_limit    17071000
final_7_limit     18094800
final_1_limit     16274900
dtype: int64

#### Diagnostics

In [12]:
# Count of missing limit reason
df_sr_scored_limits_approved[['rules_summary_narration', 'limit_reason']].isna().sum()

rules_summary_narration    6
limit_reason               6
dtype: int64

In [13]:
# Validate missing limit reason are test accounts
df_sr_scored_limits_approved.loc[(df_sr_scored_limits_approved['rules_summary_narration'].isna()) & (df_sr_scored_limits_approved['limit_reason'].isna()), 'store_number'].isin(test_accounts).sum()

6

In [14]:
# Till activity data period span
df_sr_scored_limits_approved['actual_trx_days'].describe()

count   108023.00
mean        14.82
std         11.93
min          0.00
25%          2.00
50%         14.00
75%         28.00
max         31.00
Name: actual_trx_days, dtype: float64

In [15]:
previous = pd.read_parquet(project_dir + before_21d_graduation_limits_data_path)
previous.head(2)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit
0,254714389522,772757,2022-10-09,2022-10-09,1,33.33,1,1.00,No_rules_relaxed,38.00,No,0.00,25135445,True,Approve,0.00,nan,1,601.00,30.00,12600.00,0.00,2017-12-18,2018-01-17,2018-12-31,2018-01-17,NaT,NaT,348.00,1.00,written-off_default,NaN,12600.00,2017-12-18,0.00,0.00,1794.00,0,348.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0,0.00,0.00,0,0,0,0,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,0,0.25,0.06,0.04,0.04,2.08,1.25,1.25,0.00,0.00,0.00,0,0,0,1,0
1,254723173634,786152,2022-09-20,2022-10-19,9,3843.33,30,0.30,No_rules_relaxed,28.00,No,0.00,494024,True,Approve,0.00,nan,1,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,NaT,NaT,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1794.00,0,713.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0,0.00,0.00,0,0,0,0,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 2,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0


In [16]:
after_95 = pd.read_parquet(project_dir + after_rmdd_ge95_data_path)
after_95.head(2)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit
0,254714389522,772757,2022-10-09,2022-10-09,1,33.33,1,1.00,No_rules_relaxed,38.00,No,0.00,25135445,True,Approve,0.00,nan,1,601.00,30.00,12600.00,0.00,2017-12-18,2018-01-17,2018-12-31,2018-01-17,NaT,NaT,348.00,1.00,written-off_default,NaN,12600.00,2017-12-18,0.00,0.00,1794.00,0,348.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0,0.00,0.00,0,0,0,0,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,0,0.25,0.06,0.04,0.04,2.08,1.25,1.25,0.00,0.00,0.00,0,0,1,0,0,0.00,0.00,0.00,0.00
1,254723173634,786152,2022-09-20,2022-10-19,9,3843.33,30,0.30,No_rules_relaxed,28.00,No,0.00,494024,True,Approve,0.00,nan,1,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,NaT,NaT,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1794.00,0,713.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0,0.00,0.00,0,0,0,0,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 2,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1,0,0,0.00,0.00,0.00,0.00


In [17]:
after_95[(after_95.af_final_21_limit == 0) & (after_95.af95_final_21_limit > 0)].sample(5)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit
30456,254700670239,7652485,2022-10-20,2022-10-20,1,15500.00,1,1.00,relax_rules,27.00,No,0.00,23185869,True,Reject,0.00,254700670239,14,600.00,21.00,4800.00,4800.00,2022-09-06,2022-09-27,2022-09-25,2022-09-27,2022-10-02,2022-12-31,-2.00,2.00,closed_early_repayment,443.00,13500.00,2022-06-03,14.00,1.00,71.00,1,0.00,3.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,-7.00,168500,166800.00,166600.00,11100,7200,7200,1,1,1,14,0.35,0.12,0.12,0.70,0.71,0.71,Band 7,Band 8,0.55,0.30,0.30,1.00,1.00,0,0.75,0.29,0.16,0.16,4475.62,2461.76,2461.76,4475.62,2461.76,2461.76,2500,2500,0,0,4500,0.00,4500.00,4500.00,4500.00
35526,254708837375,7097655,2022-09-20,2022-10-20,27,129933.67,31,0.87,relax_rules,27.00,No,0.00,27351754,True,Approve,4000.00,254708837375,9,600.00,7.00,500.00,500.00,2022-10-13,2022-10-20,2022-10-14,2022-10-20,2022-10-23,2023-01-21,-6.00,2.00,closed_early_repayment,481.00,6000.00,2022-07-14,9.00,1.00,34.00,1,-1.00,5.00,2.00,15.00,2.00,1.00,500.00,1500.00,0.00,750.00,-9.00,134900,94000.00,94000.00,24400,11100,11100,1,1,1,9,0.50,0.17,0.17,1.00,1.00,1.00,Band 6,Band 6,0.40,0.23,0.23,1.00,1.00,0,0.75,0.30,0.17,0.17,38980.10,21926.31,21926.31,38980.10,21926.31,21926.31,11100,11100,0,2300,24400,0.00,24400.00,24400.00,24400.00
32362,254721969210,160046,2022-09-20,2022-10-20,29,310881.02,31,0.94,relax_rules,27.00,No,0.00,11592239,True,Approve,5000.00,254721969210,29,600.00,21.00,200000.00,200000.00,2022-09-28,2022-10-19,2022-10-15,2022-10-19,2022-10-24,2023-01-22,-4.00,2.00,closed_early_repayment,480.00,200000.00,2022-09-28,29.00,1.00,49.00,1,0.00,5.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,-9.00,4046800,1873300.00,1928300.00,170700,111500,111500,1,1,1,29,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 8,0.55,0.30,0.30,1.00,1.00,0,0.75,0.41,0.22,0.22,128238.42,69948.23,69948.23,128238.42,69948.23,69948.23,70000,70000,0,0,128300,0.00,128300.00,128300.00,128300.00
30060,254712698130,778226,2022-09-20,2022-10-20,12,167256.67,31,0.39,relax_rules,27.00,No,0.00,27693501,True,Approve,4500.00,254712698130,9,600.00,21.00,6000.00,6000.00,2022-09-02,2022-09-23,2022-09-23,2022-09-23,2022-09-28,2022-12-27,0.00,2.00,closed_on_time,NaN,16000.00,2022-04-27,9.00,1.00,75.00,1,0.00,1.00,0.00,21.00,0.00,0.00,0.00,0.0

In [18]:
after_96 = pd.read_parquet(project_dir + after_rmdd_ge96_data_path)
after_96.head(2)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit
0,254714389522,772757,2022-10-09,2022-10-09,1,33.33,1,1.00,No_rules_relaxed,38.00,No,0.00,25135445,True,Approve,0.00,nan,1,601.00,30.00,12600.00,0.00,2017-12-18,2018-01-17,2018-12-31,2018-01-17,NaT,NaT,348.00,1.00,written-off_default,NaN,12600.00,2017-12-18,0.00,0.00,1794.00,0,348.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0,0.00,0.00,0,0,0,0,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,0,0.25,0.06,0.04,0.04,2.08,1.25,1.25,0.00,0.00,0.00,0,0,1,0,0,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0,0,0,0,0,0
1,254723173634,786152,2022-09-20,2022-10-19,9,3843.33,30,0.30,No_rules_relaxed,28.00,No,0.00,494024,True,Approve,0.00,nan,1,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,NaT,NaT,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1794.00,0,713.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0,0.00,0.00,0,0,0,0,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 2,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1,0,0,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0,0,0,0,0,0


In [19]:
after_96[(after_96.zeroized_1 == 1) & (after_96.due_date_rm_ge_rm_add_back == 1) & (after_96.rllvr_date_rm_ge_rm_add_back == 1)].sample(5)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit
32081,254723719337,8012145,2022-09-20,2022-10-20,28,18802.20,31,0.90,relax_rules,27.00,No,0.00,32759860,True,Approve,3500.00,254723719337,5,600.00,21.00,800.00,800.00,2022-09-24,2022-10-15,2022-10-15,2022-10-15,2022-10-20,2023-01-18,0.00,2.00,closed_on_time,466.00,800.00,2022-09-24,5.00,1.00,53.00,1,0.00,4.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,-5.00,0,15900.00,15000.00,0,3000,3000,1,1,1,5,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 4,0.35,0.20,0.20,1.00,1.00,0,0.75,0.26,0.15,0.15,4935.58,2820.33,2820.33,4935.58,2820.33,2820.33,2900,2900,0,0,0,0.00,0.00,0.00,0.00,0,0,0,2900,2900,0.00,0,0,1,1,2900,2900
31218,254724550858,881833,2022-09-20,2022-10-20,30,231312.02,31,0.97,relax_rules,27.00,No,0.00,24672610,True,Reject,0.00,254724550858,13,600.00,21.00,80000.00,80000.00,2022-09-14,2022-10-05,2022-10-05,2022-10-05,2022-10-10,2023-01-08,0.00,2.00,closed_on_time,472.00,80700.00,2022-07-04,13.00,1.00,63.00,1,2.00,3.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,-5.00,1280300,1404400.00,1299100.00,121200,69000,69000,1,1,1,13,0.35,0.12,0.12,0.70,0.71,0.71,Band 7,Band 8,0.55,0.30,0.30,1.00,1.00,0,0.75,0.29,0.16,0.16,66791.34,36737.79,36737.79,66791.34,36737.79,36737.79,36800,36800,0,0,66800,0.00,66800.00,66800.00,66800.00,0,0,66800,36800,36800,66800.00,0,0,1,1,36800,36800
31439,254727660643,7151794,2022-09-20,2022-10-20,30,67594.21,31,0.97,relax_rules,27.00,No,0.00,22852239,True,Approve,5000.00,254727660643,4,600.00,21.00,10200.00,10200.00,2022-09-16,2022-10-07,2022-10-07,2022-10-07,2022-10-12,2023-01-10,0.00,2.00,closed_on_time,466.00,10200.00,2022-09-16,4.00,1.00,61.00,1,0.00,1.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,-5.00,114400,263700.00,260600.00,19400,12700,12700,1,1,1,4,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,15208.70,8871.74,8871.74,15208.70,8871.74,8871.74,8900,8900,0,0,15300,0.00,15300.00,15300.00,15300.00,0,0,15300,8900,8900,15300.00,0,0,1,1,8900,8900
30455,254797890141,7588143,2022-09-22,2022-10-18,12,24763.33,27,0.